In [9]:
import numpy as np

def get_psi_1(ket_0, ket_1, h_gate):
    """
    Step 1: Initialize |0>|1> and apply Hadamard to both.
    Returns: |psi_1>
    """
    # Apply H to each qubit individually
    h_ket_0 = h_gate @ ket_0
    h_ket_1 = h_gate @ ket_1

    # Tensor them together: |psi_1> = (H|0>) (tensor) (H|1>)
    psi_1 = np.kron(h_ket_0, h_ket_1)

    # PRINTING PSI_1 AS REQUESTED
    print(f"State after first Hadamards |psi_1>:\n{psi_1.flatten().round(3)}")
    return psi_1

def create_oracle_matrix(f):
    """
    Constructs the 4x4 Unitary Matrix U_f for a given function f.
    Mapping: |x, y> -> |x, y XOR f(x)>
    """
    U_f = np.zeros((4, 4))
    for i in range(4):
        # Decode index i -> x, y
        x = (i >> 1) & 1  # Top bit
        y = i & 1         # Bottom bit

        # Apply function logic
        y_new = y ^ f(x)

        # Encode x, y_new -> index j
        j = (x << 1) | y_new
        U_f[j, i] = 1
    return U_f

def apply_final_hadamard(psi_2, h_gate, i_gate):
    """
    Step 3: Apply Hadamard to the Top Qubit only.
    Operator: H (tensor) I
    """
    final_op = np.kron(h_gate, i_gate)
    psi_3 = final_op @ psi_2
    return psi_3

def measure_state(psi_3):
    """
    Step 4: Calculate probabilities for the Top Qubit.
    """
    # Probability of Top Qubit = 0 is sum of squares of first two amplitudes (|00>, |01>)
    prob_top_0 = np.abs(psi_3[0])**2 + np.abs(psi_3[1])**2

    # Probability of Top Qubit = 1 is sum of squares of last two amplitudes (|10>, |11>)
    prob_top_1 = np.abs(psi_3[2])**2 + np.abs(psi_3[3])**2

    return prob_top_0, prob_top_1

def run_deutsch_simulation(name, f, ket_0, ket_1, h_gate, i_gate):
    """
    Orchestrates the full Deutsch Algorithm for a specific function f.
    """
    print(f"\n{'='*60}")
    print(f"--- Simulation: {name} ---")
    print(f"Values: f(0)={f(0)}, f(1)={f(1)}")

    # 1. Get Initial Superposition
    psi_1 = get_psi_1(ket_0, ket_1, h_gate)

    # 2. Apply Oracle
    U_f = create_oracle_matrix(f)
    psi_2 = U_f @ psi_1

    # 3. Apply Interference (Final Hadamard)
    psi_3 = apply_final_hadamard(psi_2, h_gate, i_gate)

    # 4. Measure
    prob_0, prob_1 = measure_state(psi_3)

    print(f"Final State Vector:\n{psi_3.flatten().round(3)}")
    print(f"Prob(Top=0): {prob_0[0]:.2f}")
    print(f"Prob(Top=1): {prob_1[0]:.2f}")

    if np.isclose(prob_0, 1.0):
        print(">> MEASUREMENT: 0")
        print(">> CONCLUSION: Function is CONSTANT.")
    elif np.isclose(prob_1, 1.0):
        print(">> MEASUREMENT: 1")
        print(">> CONCLUSION: Function is BALANCED.")
    else:
        print(">> ERROR: Inconclusive measurement.")

def main():
    # --- Configuration & Constants ---
    # Moved inside main() to avoid global scope pollution

    # |0> and |1>
    ket_0 = np.array([[1], [0]])
    ket_1 = np.array([[0], [1]])

    # Gates
    h_gate = (1 / np.sqrt(2)) * np.array([[1, 1], [1, -1]])
    i_gate = np.array([[1, 0], [0, 1]])

    # Define the 4 Functions
    functions = [
        ("Constant Zero (f(x)=0)", lambda x: 0),
        ("Constant One (f(x)=1)", lambda x: 1),
        ("Balanced Identity (f(x)=x)", lambda x: x),
        ("Balanced Not (f(x)=NOT x)", lambda x: 1 - x)
    ]

    for name, func in functions:
        run_deutsch_simulation(name, func, ket_0, ket_1, h_gate, i_gate)

if __name__ == "__main__":
    main()


--- Simulation: Constant Zero (f(x)=0) ---
Values: f(0)=0, f(1)=0
State after first Hadamards |psi_1>:
[ 0.5 -0.5  0.5 -0.5]
Final State Vector:
[ 0.707 -0.707  0.     0.   ]
Prob(Top=0): 1.00
Prob(Top=1): 0.00
>> MEASUREMENT: 0
>> CONCLUSION: Function is CONSTANT.

--- Simulation: Constant One (f(x)=1) ---
Values: f(0)=1, f(1)=1
State after first Hadamards |psi_1>:
[ 0.5 -0.5  0.5 -0.5]
Final State Vector:
[-0.707  0.707  0.     0.   ]
Prob(Top=0): 1.00
Prob(Top=1): 0.00
>> MEASUREMENT: 0
>> CONCLUSION: Function is CONSTANT.

--- Simulation: Balanced Identity (f(x)=x) ---
Values: f(0)=0, f(1)=1
State after first Hadamards |psi_1>:
[ 0.5 -0.5  0.5 -0.5]
Final State Vector:
[ 0.     0.     0.707 -0.707]
Prob(Top=0): 0.00
Prob(Top=1): 1.00
>> MEASUREMENT: 1
>> CONCLUSION: Function is BALANCED.

--- Simulation: Balanced Not (f(x)=NOT x) ---
Values: f(0)=1, f(1)=0
State after first Hadamards |psi_1>:
[ 0.5 -0.5  0.5 -0.5]
Final State Vector:
[ 0.     0.    -0.707  0.707]
Prob(Top=0): 0.00